#### 1. import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

In [2]:
df = pd.read_csv("Final_cleaned_code_dataset.csv")
df.head()

,code,complexity,tags,code_length
0,"from math import sqrt a, v = map(int, input()....",constant,"['implementation', 'math']",509
1,"from math import * a, vm = map(int, input().sp...",constant,"['implementation', 'math']",614
2,"import os import sys from io import BytesIO, I...",constant,"['implementation', 'math']",1999
3,"from math import * a, vm = map(int, input().sp...",constant,"['implementation', 'math']",614
4,"from math import * a,v=list(map(int,input().sp...",constant,"['implementation', 'math']",432


#### 2. Encode Complexity

In [3]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_tc= label_encoder.fit_transform(df['complexity'])
print("TC classes:", label_encoder.classes_)

TC classes: ['constant' 'cubic' 'linear' 'np' 'quadratic']


#### 3. Encode topics (multi-label)

In [4]:
import ast
def parse_topics(topic_str):
    try:
        #  string of list "['implementation', 'math']" to actual Python list
        topics = ast.literal_eval(topic_str)
        # Make lowercase and strip spaces
        topics = [t.strip().lower() for t in topics if t.strip()]
        return topics
    except:
        return []

df['tags'] = df['tags'].apply(parse_topics)

In [5]:
mlb = MultiLabelBinarizer()
y_topics = mlb.fit_transform(df['tags'])
print("Topics classes:", mlb.classes_)

Topics classes: ['2-sat' 'binary search' 'bitmasks' 'brute force' 'combinatorics'
 'constructive algorithms' 'data structures' 'dfs and similar'
 'divide and conquer' 'dp' 'dsu' 'expression parsing' 'flows' 'games'
 'geometry' 'graph matchings' 'graphs' 'greedy' 'hashing' 'implementation'
 'interactive' 'math' 'meet-in-the-middle' 'number theory' 'probabilities'
 'shortest paths' 'sortings' 'strings' 'trees' 'two pointers']


#### 4. TF-IDF for code

In [6]:
tfidf = TfidfVectorizer(max_features=3000)
X_code = tfidf.fit_transform(df['code'])
print("Code TF-IDF shape:", X_code.shape)

Code TF-IDF shape: (3309, 3000)


#### 5. Combine code + tags features as input

In [7]:
from scipy.sparse import hstack

X = hstack([X_code, y_topics])
print("Combined feature shape:", X.shape)

# 3000 col from code(TF-IDF) and 30 from tags

Combined feature shape: (3309, 3030)


#### 6. Train-test split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_code, y_tc, test_size=0.2, random_state=42, stratify=y_tc)
print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

# 80% train and 20% test

Train shape: (2647, 3000) Test shape: (662, 3000)


#### 7. Logistic Regression

In [9]:
log_reg = LogisticRegression(max_iter=500, class_weight='balanced')
log_reg.fit(X_train, y_train)
y_pred_lr = log_reg.predict(X_test)

print("=== Logistic Regression ===")
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))

=== Logistic Regression ===
Accuracy: 0.6253776435045317
              precision    recall  f1-score   support

           0       0.83      0.82      0.83       154
           1       0.65      0.66      0.66       115
           2       0.52      0.47      0.50       165
           3       0.70      0.59      0.64       100
           4       0.47      0.59      0.52       128

    accuracy                           0.63       662
   macro avg       0.64      0.63      0.63       662
weighted avg       0.63      0.63      0.63       662



#### 8. MLP Classifier

In [10]:
mlp = MLPClassifier(hidden_layer_sizes=(256,128), activation='relu', max_iter=500)
mlp.fit(X_train, y_train)
y_pred_mlp = mlp.predict(X_test)

print("=== MLP Classifier ===")
print("Accuracy:", accuracy_score(y_test, y_pred_mlp))
print(classification_report(y_test, y_pred_mlp))

=== MLP Classifier ===
Accuracy: 0.620845921450151
              precision    recall  f1-score   support

           0       0.76      0.75      0.76       154
           1       0.72      0.62      0.66       115
           2       0.51      0.57      0.54       165
           3       0.70      0.64      0.67       100
           4       0.49      0.52      0.50       128

    accuracy                           0.62       662
   macro avg       0.64      0.62      0.63       662
weighted avg       0.63      0.62      0.62       662



In [32]:
  ## Saving the models
import pickle

with open("branch1_log_reg.pkl", "wb") as f:
    pickle.dump(log_reg, f)

with open("branch1_mlp.pkl", "wb") as f:
    pickle.dump(mlp, f)

with open("branch1_tfidf.pkl", "wb") as f:
    pickle.dump(tfidf, f)

In [34]:
  ## Loading the models
import pickle

with open("branch1_log_reg.pkl", "rb") as f:
    log_reg = pickle.load(f)

with open("branch1_mlp.pkl", "rb") as f:
    mlp = pickle.load(f)

with open("branch1_tfidf.pkl", "rb") as f:
    tfidf = pickle.load(f)

In [35]:
new_code = """
print('Hello, world!')
"""

In [36]:
X_new = tfidf.transform([new_code])

# Predict time complexity
predicted_index = log_reg.predict(X_new)[0]

# Map index to actual label
predicted_tc = label_encoder.classes_[predicted_index]

print("Predicted Time Complexity:", predicted_tc)

Predicted Time Complexity: constant


In [37]:
new_code = """
def sum_array(arr):
    total = 0
    for i in arr:
        total += i
    return total
"""

In [38]:
X_new = tfidf.transform([new_code])

# Predict time complexity
predicted_index = mlp.predict(X_new)[0]

# Map index to actual label
predicted_tc = label_encoder.classes_[predicted_index]

print("Predicted Time Complexity:", predicted_tc)

Predicted Time Complexity: quadratic


#### Limitations of Branch 1

1. Limited Semantic Understanding
    Branch 1 uses TF-IDF vectorization of code, which only captures statistical patterns of tokens.
    It does not understand loops, recursion, or nesting beyond simple patterns.

2. Misclassification of Complex Code
    Works well for simple code snippets (single loops, sequential statements).
    Fails for nested loops, recursion, or more intricate logic, often predicting incorrect TC.

3. Ambiguous Class ‘np’
    Some predictions fall into the ‘np’ class, which represents not predictable / unclear codes.
    This indicates that the model cannot confidently classify certain code snippets.

4. Motivation for Branch 2
    These limitations highlight the need for a semantic code understanding model like CodeT5, which can learn the structural and logical patterns of code for more accurate time complexity predictions.